# Graph visualization

This notebook demonstrates the use of the TrustGraph triple store API to fetch information from the knowledge graph. This information is then visualized using the networkX graph visualisation library.  This notebook was executed against a running TrustGraph loaded with the space shuttle challenger report from NASA located here:

https://ntrs.nasa.gov/citations/19860015255

In [1]:
import requests
import json
import base64

In [3]:
# URL of the TrustGraph triplestore query API
url = "http://localhost:8088/api/v1/load/text"

In [2]:
text = open("../sources/README.cats", "rb").read()

In [4]:
org_id = "https://trustgraph.ai/org/3c35111a-f8ce-54b2-4dd6-c673f8bf0d09"
doc_id = "https://trustgraph.ai/doc/4faa45c1-f91a-a96a-d44f-2e57b9813db8"
pub_id = "https://trustgraph.ai/pubev/a847d950-a281-4099-aaab-c5e35333ff61"

In [5]:
org_facts = [
    [org_id, "http://www.w3.org/1999/02/22-rdf-syntax-ns#type", "https://schema.org/Organization"],
    [org_id, "http://www.w3.org/2000/01/rdf-schema#label", "trustgraph.ai"],
    [org_id, "https://schema.org/name", "trustgraph.ai"]
]

In [6]:
pub_facts = [
    [pub_id, "http://www.w3.org/1999/02/22-rdf-syntax-ns#type", "https://schema.org/PublicationEvent"],
    [pub_id, "https://schema.org/description", "Uploading to Github"],
    [pub_id, "https://schema.org/endDate", "2024-10-23"],
    [pub_id, "https://schema.org/publishedBy", org_id],
    [pub_id, "https://schema.org/startDate", "2024-10-23"]
]

In [7]:
doc_facts = [
    [doc_id, "http://www.w3.org/1999/02/22-rdf-syntax-ns#type", "https://schema.org/DigitalDocument"],
    [doc_id, "http://www.w3.org/2000/01/rdf-schema#label", "Mark's cats"],
    [doc_id, "https://schema.org/copyrightHolder", "trustgraph.ai"],
    [doc_id, "https://schema.org/copyrightNotice", "Public domain"],
    [doc_id, "https://schema.org/copyrightYear", "2024"],
    [doc_id, "https://schema.org/description", "This document describes Mark's cats"],
    [doc_id, "https://schema.org/keywords", "animals"],
    [doc_id, "https://schema.org/keywords", "cats"],
    [doc_id, "https://schema.org/keywords", "home-life"],
    [doc_id, "https://schema.org/name", "Mark's cats"],
    [doc_id, "https://schema.org/publication", pub_id],
    [doc_id, "https://schema.org/url", "https://example.com"]
]

In [8]:
metadata = [
    { "s": t[0], "p": t[1], "o": t[2] }
    for t in org_facts + pub_facts + doc_facts
]

In [9]:
input = {
    "id": doc_id,
    "metadata": metadata,
    "charset": "utf-8",
    "text": base64.b64encode(text).decode("utf-8")
}

In [10]:
resp = requests.post(url, json=input)

In [11]:
# Should be a 200 status code
resp.status_code

200